**I import some useful libraries in python.**

In [ ]:
import json
import csv
!pip install unidecode
from unidecode import unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.6 MB/s eta 0:00:00


**I mount google drive because all the files (for example .csv) that i used for the implementation of my thesis are saved on my google drive.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Reading the training and validation datasets that are in .txt file format and droping of the column of their answers ids.**

In [ ]:
import pandas as pd
train_dataset = pd.read_csv('annotated_wd_data_train.txt', sep="\t", names=['entity_id','relation_id', 'answer', 'Question']) #read train dataset
validation_dataset = pd.read_csv('annotated_wd_data_valid.txt', sep="\t", names=['entity_id','relation_id', 'answer', 'Question'])#read validation dataset
train_dataset = train_dataset.drop(labels='answer', axis=1) #drop answer
validation_dataset = validation_dataset.drop(labels='answer', axis=1) #drop answer

**I made some preprocessing for each question by creating the function preprocessing, which takes as argument the text (in this case the question) and performs accent removal, as well as punctuation mark removal (period, question mark, exclamation mark), s removal with apostrophe('s) and comma(,) removal.Finally this function, which is shown below, returns the original text with the above mentioned processing.**

In [ ]:
def Preprocessing(text):
    text = unidecode(text.replace("?", "").replace("'s", "").replace(".", "").replace("!", "").replace(",", ""))
    return text

train_dataset['Question'] = train_dataset['Question'].apply(lambda x:Preprocessing(x))
validation_dataset['Question'] = validation_dataset['Question'].apply(lambda x:Preprocessing(x))

**The SPARQLWrapper library is downloaded, which contributes to the generation of SPARQL queries, in order to find the entity labels.**

In [ ]:
#agent='WDQS-example Python'
#Chicobot-Agent
!pip install sparqlwrapper
import sys
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent= "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1]))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00


**The function find\_divisors which receives as an argument the length of either the SimpleQuestions training or the SimpleQuestions validation dataset and returns the divisors of this number, has as aim to find the divisors of the lengths of of the full SimpleQuestions training and validation dataset, so that it is then possible to get the entity labels from SPARQL as quickly as possible.**

In [ ]:
def find_divisors(a):
    divisors = []
    for i in range(1, a + 1):
        if a % i == 0:
            divisors.append(i)
    return divisors

# Example usage
number_to_check_1 = 34374
number_to_check_2 = 4867
result_1 = find_divisors(number_to_check_1)
result_2 = find_divisors(number_to_check_2)
print(f"The divisors of {number_to_check_1} are: {result_1}")
print(f"The divisors of {number_to_check_2} are: {result_2}")

The divisors of 34374 are: [1, 2, 3, 6, 17, 34, 51, 102, 337, 674, 1011, 2022, 5729, 11458, 17187, 34374]
The divisors of 4867 are: [1, 31, 157, 4867]


**The SPARQL Queries below found from wikidata the entity labels and the entity ids for each entity id (of course for each question) of the SimpleQuestions training και validation dataset.**

In [ ]:
train_entity_names = []
train_entity_ids = []
for i in range(0,len(train_dataset),102):  #retrieve training set entities' corresponding ids
  str = ""
  for entity_id in train_dataset['entity_id'][i:i+102]:
   str = str + "wd:" + entity_id + " "
  sparql.setQuery("""
  SELECT ?item ?itemLabel
  WHERE
  {
    VALUES ?item {""" + str + """}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
""")
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  train_results_df = pd.json_normalize(results['results']['bindings'])
  for j in train_results_df['item.value']:
      train_entity_ids.append(j)
  for j in train_results_df['itemLabel.value']:
      train_entity_names.append(j)

In [ ]:
validation_entity_names = []
validation_entity_ids = []
for i in range(0,len(validation_dataset),157):  #retrieve validation set entity's corresponding ids
  str = ""
  for entity_id in validation_dataset['entity_id'][i:i+157]:
   str = str + "wd:" + entity_id + " "
  sparql.setQuery("""
  SELECT ?item ?itemLabel
  WHERE
  {
    VALUES ?item {""" + str + """}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
""")
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  validation_results_df = pd.json_normalize(results['results']['bindings'])
  for j in validation_results_df['item.value']:
      validation_entity_ids.append(j)
  for j in validation_results_df['itemLabel.value']:
      validation_entity_names.append(j)

**Creation of the corresponding lists containing the questions, the entity labels, the relation ids and the entity ids of the corresponding SimpleQuestions training and validation dataset, either this dataset includes answerable and answerable questions over wikidata questions or only answerable questions over wikidata.**

In [ ]:
train_dataset_questions = train_dataset['Question'].to_list()
validation_dataset_questions = validation_dataset['Question'].to_list()
train_dataset_entity_ids = train_dataset['entity_id'].to_list()
validation_dataset_entity_ids = validation_dataset['entity_id'].to_list()

**The entity ids that are occurred from SPARQL are expressions that contain an encoded name, for example for the question of the SimpleQuestions (full) training dataset "who is a musician born in detroit", the entity id that occurred is https://www.wikidata.org/wiki/Q12439, which is related with Detroit (Q12439, if i search in wikidata).**

**So the python function entity\_ids is created with the aim to split the full expression with the entity id, such as the example above and to keep the encoded name, i.e named start from "Q" ("Q12439" in the above example).**

In [ ]:
#Split Spaqrl URI's in order to get the entity ids
import os
def enitity_ids(ids): #get ids from sparql query ids
 entity_ids = []
 for i in range(len(ids)):
   entity_ids.append(ids[i].split(os.path.sep)[-1])
 return entity_ids
train_ids = enitity_ids(train_entity_ids)
validation_ids = enitity_ids(validation_entity_ids)
print(len(train_ids))
print(len(train_dataset['entity_id']))
print(len(validation_dataset['entity_id']))
print(len(validation_ids))


34852
34374
4867
4983


**The function labels from which i confirm the entity id of each question, which occurred from SPARQL, through the entity id obtained for each question from the transfer of SimpleQuestions from Freebase to Wikidata. This function takes as arguments the list of questions, the list of the entity ids both from the corresponding SimpleQuestions dataset column and from the entity\_ids function and the list of entity labels occured from SPARQL.**

**Essentially i assigned to each question and entity id its corresponding entity label by searching in the list of training and validation entity ids obtained from SPARQL. If any of these entity ids exist in the SimpleQuestions training and validation entity labels lists, that are created before and matching the entity label and thus a new column with entity labels for the training and the validation dataset.**

In [ ]:
import operator as op
def labels(questions, entity_id_dataset, entity_ids, entity_names):# entity_ids and dataset must have the same length
#In this step we find the entity ids from the initial dataset to the list occured from Sparql Query
#for each of the train and validation dataset.
#This will become the train or validation data new column, containing entities' labels

  label_df_column = []
  for entity_id in entity_id_dataset:
   j=0
   for ent in entity_ids: #find its id on list
    if entity_id == ent:
      label_df_column.append(entity_names[j]) #and find corresponding label
      break
    j += 1
  return label_df_column


train_dataset['entity_label'] = labels(train_dataset_questions, train_dataset_entity_ids, train_ids, train_entity_names)
validation_dataset['entity_label'] = labels(validation_dataset_questions, validation_dataset_entity_ids ,validation_ids, validation_entity_names)
print(train_dataset['entity_label'], len(train_dataset['entity_label']))
print(validation_dataset['entity_label'], len(validation_dataset['entity_label']))

0            Warner Bros.
1                 Detroit
2                Q7370831
3             Mera Shikar
4                 Chicago
               ...       
34369      Anthony Bailey
34370      Homi K. Bhabha
34371    video game music
34372    Gastón Filgueira
34373            defender
Name: entity_label, Length: 34374, dtype: object 34374
0           JW Marriott Panama
1                Sasha Vujačić
2            Wiebke Carolsfeld
3       Seymour Parker Gilbert
4             Antoine de Févin
                 ...          
4862            Herby Fortunat
4863               The Pianist
4864                  Jon Seda
4865         Nikolaj Frobenius
4866                 São Paulo
Name: entity_label, Length: 4867, dtype: object 4867


**The preprocessing of the entity labels column occurred from labels function is made through function Preprocessing that is created previously. The reindexing of the columns (entity id, relation id, questions, entity label) of the initial SimpleQuestions dataset (either full or the dataset containing only questions that are answerable over wikidata),in order to include the entity labels as a new column in the initial SimpleQuestions dataset, is achieved by using the reindex() method from pandas.**

In [ ]:
train_dataset['entity_label'] = train_dataset['entity_label'].apply(lambda x:Preprocessing(x))
validation_dataset['entity_label'] = validation_dataset['entity_label'].apply(lambda x:Preprocessing(x))
new_columns = ["entity_id","entity_label","relation_id","Question"]
train_dataset = train_dataset.reindex(columns=new_columns)
validation_dataset = validation_dataset.reindex(columns=new_columns)
print(train_dataset)
print(validation_dataset)

       entity_id      entity_label relation_id  \
0        Q126399       Warner Bros        R272   
1         Q12439           Detroit         R19   
2       Q7370831          Q7370831        P162   
3       Q6817891       Mera Shikar        P364   
4          Q1297           Chicago        R276   
...          ...               ...         ...   
34369  Q16093542    Anthony Bailey         P27   
34370    Q325741     Homi K Bhabha        P737   
34371   Q1062702  video game music        R136   
34372    Q926822  Gaston Filgueira         P21   
34373    Q336286          defender        R413   

                                                Question  
0                  what movie is produced by warner bros  
1                      who is a musician born in detroit  
2                who produced the film rough house rosie  
3      what is the language in which mera shikar was ...  
4      Whats the name of a battle that happened in ch...  
...                                          

**Creation of the corresponding .csv files for the SimpleQuestions training and validation dataset respectively through pandas method to_csv().**

In [ ]:
import csv
# Specify the file name
train_csv_name = "drive/MyDrive/train_dataset.csv"
# Writing to CSV file
train_dataset.to_csv(train_csv_name, index=False)

print(train_dataset)

       entity_id      entity_label relation_id  \
0        Q126399       Warner Bros        R272   
1         Q12439           Detroit         R19   
2       Q7370831          Q7370831        P162   
3       Q6817891       Mera Shikar        P364   
4          Q1297           Chicago        R276   
...          ...               ...         ...   
34369  Q16093542    Anthony Bailey         P27   
34370    Q325741     Homi K Bhabha        P737   
34371   Q1062702  video game music        R136   
34372    Q926822  Gaston Filgueira         P21   
34373    Q336286          defender        R413   

                                                Question  
0                  what movie is produced by warner bros  
1                      who is a musician born in detroit  
2                who produced the film rough house rosie  
3      what is the language in which mera shikar was ...  
4      Whats the name of a battle that happened in ch...  
...                                          

In [ ]:
valid_csv_name = "drive/MyDrive/valid_dataset.csv"
validation_dataset.to_csv(valid_csv_name, index=False)


print(validation_dataset)

     entity_id            entity_label relation_id  \
0     Q3541144      JW Marriott Panama        P138   
1      Q318926           Sasha Vujacic         P19   
2     Q2568216       Wiebke Carolsfeld         R57   
3     Q2275923  Seymour Parker Gilbert        P106   
4     Q2856873        Antoine de Fevin         P20   
...        ...                     ...         ...   
4862  Q3709870          Herby Fortunat        P413   
4863   Q150804             The Pianist        P136   
4864  Q1343857                Jon Seda         P19   
4865  Q2981169       Nikolaj Frobenius        P106   
4866      Q174               Sao Paulo         R19   

                                               Question  
0     Who was the trump ocean club international hot...  
1                          where was sasha vujacic born  
2          What is a film directed by wiebke carolsfeld  
3            What was Seymour Parker Gilbert profession  
4         in what french city did antoine de fevin die   
...